In [1]:
from pathlib import Path
from utils.base_utils import read_yaml

yaml_path = Path("../config/config.yaml")

try:
    config = read_yaml(yaml_path)
    print(config)
except Exception as e:
    print(f"Error: {e}")

2025-06-18 14:25:14,974 | fsd_logger | INFO | base_utils.py:39 | YAML file ../config/config.yaml loaded successfully
{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://drive.google.com/file/d/1ORlqNeY3OnD7nHNDJ_NTN93QvyurvZvh/view?usp=sharing', 'local_datafile': 'artifacts/data_ingestion/fsd.zip', 'unzip_dir': 'artifacts/data_ingestion'}}


In [2]:
from pathlib import Path
from utils.base_utils import create_directories

base_path = Path("../test")
dirs = [base_path / "dir1", base_path / "dir2"]
create_directories(dirs)

2025-06-18 14:25:14,983 | fsd_logger | INFO | base_utils.py:63 | Directory created at ../test/dir1
2025-06-18 14:25:14,986 | fsd_logger | INFO | base_utils.py:63 | Directory created at ../test/dir2


### Initial Testing

In [3]:
from dataclasses import dataclass
from pathlib import Path
from constants import *
from utils.base_utils import read_yaml, create_directories
import os
import zipfile
import gdown
from utils import logger

In [4]:
@dataclass(frozen=True)
class DataIngestionConfig:
   root_dir: Path
   source_URL: str
   local_datafile: Path
   unzip_dir: Path

In [5]:
class ConfigurationManager:

    def __init__(self, config_filepath = CONFIG_FILE_PATH, param_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.param = read_yaml(param_path)
        
        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([self.config.artifacts_root])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_datafile = config.local_datafile ,
            unzip_dir = config.unzip_dir
        )
        return data_ingestion_config

In [6]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self)-> str:
        try:
            dataset_url = self.config.source_URL
            zip_donwload_dir = self.config.local_datafile
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Donwloaded data from {dataset_url} into file {zip_donwload_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix_url = "https://drive.google.com/uc?/export=download&id="
            gdown.download(prefix_url+file_id, zip_donwload_dir)
        
        except Exception as e:
            raise e
    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_datafile, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [7]:
# Pipeleline
try:
    # Initilize the ConfigurationManager
    config = ConfigurationManager()
    # Get the config yaml file details 
    data_ingestion_config = config.get_data_ingestion_config()
    # Initilize the DataIngestion
    data_ingestion = DataIngestion(config=data_ingestion_config)
    # Call download file
    data_ingestion.download_file()
    # Call unzip file
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

2025-06-18 14:25:15,134 | fsd_logger | INFO | base_utils.py:39 | YAML file ../config/config.yaml loaded successfully
2025-06-18 14:25:15,138 | fsd_logger | INFO | base_utils.py:39 | YAML file ../params.yaml loaded successfully
2025-06-18 14:25:15,140 | fsd_logger | INFO | base_utils.py:63 | Directory created at artifacts
2025-06-18 14:25:15,140 | fsd_logger | INFO | base_utils.py:63 | Directory created at artifacts
2025-06-18 14:25:15,141 | fsd_logger | INFO | 1869071566.py:10 | Donwloaded data from https://drive.google.com/file/d/1ORlqNeY3OnD7nHNDJ_NTN93QvyurvZvh/view?usp=sharing into file artifacts/data_ingestion/fsd.zip


Downloading...
From: https://drive.google.com/uc?/export=download&id=1ORlqNeY3OnD7nHNDJ_NTN93QvyurvZvh
To: /Volumes/Macintosh/mlflow-dvc-food-spoilage-detector/notebooks/artifacts/data_ingestion/fsd.zip
100%|██████████| 946/946 [00:00<00:00, 805kB/s]
